<a href="https://colab.research.google.com/github/dylan518/Stop-Layer-Implementation/blob/main/Stop_layer_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Transformations for the input data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Loading CIFAR-10 datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)


In [ ]:
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim  # This line is missing in your original snippet


class STEFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # This simulates the binary decision; actual implementation might vary
        return (input > 0.5).float()

    @staticmethod
    def backward(ctx, grad_output):
        # Pass the gradient through unchanged
        return grad_output
ste = STEFunction.apply

import torch
import torch.nn as nn
import torch.nn.functional as F

def eval(model):
  correct = 0
  total = 0
  with torch.no_grad():  # No gradients needed
      for images, labels in testloader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  accuracy = 100 * correct / total
  return accuracy

class STEFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # Simulates a binary decision; the actual implementation might vary
        return (input > 0.5).float()

    @staticmethod
    def backward(ctx, grad_output):
        # Pass the gradient through unchanged
        return grad_output

class CNNWithIterativeStopNet(nn.Module):
    def __init__(self):
        super(CNNWithIterativeStopNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # Input channels = 3 for RGB images, 6 output channels, kernel size = 5
        self.pool = nn.MaxPool2d(2, 2)   # 2x2 Max pooling
        self.conv2 = nn.Conv2d(6, 16, 5)  # Increase to 16 output channels for the second conv layer, kernel size = 5

        # Iterative Layer - processes flattened conv output
        self.iterative_fc = nn.Linear(400, 120)  # Assuming flattening to 400 features before this layer

        # Transition layer - reshapes output for potential reiteration or stop decision
        self.transition_fc = nn.Linear(120, 400)

        # StopNet (Stop Iterator Layer) - decides whether to continue iterations
        self.stop_decision_layer = nn.Linear(400, 1)  # Outputs a single value for stop decision

        # Layers after iterative processing
        self.fc2 = nn.Linear(400, 84)  # Processes output after stop decision
        self.fc3 = nn.Linear(84, 10)   # Final layer for classifying into 10 categories

    def forward(self, x):
        # First, pass x through the initial convolutional and pooling layers
        x = self.pool(F.relu(self.conv1(x)))  # First convolution then pooling
        x = self.pool(F.relu(self.conv2(x)))  # Second convolution then pooling

        # At this point, x has gone through the initial layers and should be the correct shape
        # for flattening and further processing

        # Now flatten x before passing it through the iterative process
        x = torch.flatten(x, start_dim=1)

        # Initialize the mask for the iterative process
        mask = torch.ones(x.size(0), device=x.device, dtype=torch.bool)

        max_iterations = 20
        for i in range(max_iterations):
            # Now x is correctly shaped for the iterative_fc layer
            x_iterative = F.relu(self.iterative_fc(x))
            x_trans = self.transition_fc(x_iterative)

            # Stop decision based on the current state
            tensor_stop=torch.sigmoid(self.stop_decision_layer(x_trans))

            stop_decision =tensor_stop > 0.5


            # Update the mask - False for items that should stop
            mask = mask & ~stop_decision.squeeze()


            # Use the mask to selectively apply updates
            x = torch.where(mask.unsqueeze(1), x_trans, x)

            if not mask.any():
                break  # Exit if all items have met their stop condition
        # Processing using the last state of x for each item
        if random.random() < 0.001:
          print(i)
        x_final = F.relu(self.fc2(x))
        x_final = self.fc3(x_final)
        return x_final


cnn_with_iterative_stopnet = CNNWithIterativeStopNet()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn_with_iterative_stopnet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_with_iterative_stopnet.parameters(), lr=0.0036, momentum=0.91)

num_epochs = 20  # Define the number of epochs
N = 64

for epoch in range(num_epochs):  # Outer loop over epochs
    accumulated_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()  # Move zero_grad here
        outputs = cnn_with_iterative_stopnet(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        accumulated_loss += loss.item()

        if (i + 1) % N == 0:  # N is still 64; this condition is true every iteration with batch_size=64
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}], Loss: {accumulated_loss / N:.3f}')
            if( (i + 1) % (N*5) == 0):
              print(f"accuracy {eval(cnn_with_iterative_stopnet)}")
            accumulated_loss = 0.0  # Reset for the next set of N samples

print('Finished Training')







In [ ]:
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim  # This line is missing in your original snippet


class STEFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # This simulates the binary decision; actual implementation might vary
        return (input > 0.5).float()

    @staticmethod
    def backward(ctx, grad_output):
        # Pass the gradient through unchanged
        return grad_output
ste = STEFunction.apply

import torch
import torch.nn as nn
import torch.nn.functional as F

def eval(model):
  correct = 0
  total = 0
  with torch.no_grad():  # No gradients needed
      for images, labels in testloader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  accuracy = 100 * correct / total
  return accuracy

class STEFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # Simulates a binary decision; the actual implementation might vary
        return (input > 0.5).float()

    @staticmethod
    def backward(ctx, grad_output):
        # Pass the gradient through unchanged
        return grad_output
class CNNWithIterativeStopNet(nn.Module):
    def __init__(self):
        super(CNNWithIterativeStopNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 5)  # Input channels = 3 for RGB images, 6 output channels, kernel size = 5
        self.pool = nn.MaxPool2d(2, 2)  # 2x2 Max pooling
        self.conv2 = nn.Conv2d(6, 32, 5)  # Increase to 16 output channels for the second conv layer, kernel size = 5

        # Iterative Layer - processes flattened conv output
        self.iterative_fc = nn.Linear(800, 120)  # Assuming flattening to 400 features before this layer
        self.iterative_pool = nn.MaxPool1d(2)  # 1D Max pooling with kernel size 2

        # Transition layer - reshapes output for potential reiteration or stop decision
        self.transition_fc = nn.Linear(120, 800)  # Adjusted input size to match pooled output

        # StopNet (Stop Iterator Layer) - decides whether to continue iterations
        self.stop_decision_layer = nn.Linear(800, 1)  # Outputs a single value for stop decision

        # Layers after iterative processing
        self.fc2 = nn.Linear(800, 84)  # Processes output after stop decision
        self.fc3 = nn.Linear(84, 10)  # Final layer for classifying into 10 categories

    def forward(self, x):
        # First, pass x through the initial convolutional and pooling layers
        x = self.pool(F.relu(self.conv1(x)))  # First convolution then pooling
        x = self.pool(F.relu(self.conv2(x)))  # Second convolution then pooling

        # At this point, x has gone through the initial layers and should be the correct shape
        # for flattening and further processing

        # Now flatten x before passing it through the iterative process
        x = torch.flatten(x, start_dim=1)

        # Initialize the mask for the iterative process
        mask = torch.ones(x.size(0), device=x.device, dtype=torch.bool)

        max_iterations = 10
        for i in range(max_iterations):
            # Now x is correctly shaped for the iterative_fc layer
            x_iterative = F.relu(self.iterative_fc(x))

            # Apply pooling to the output of iterative_fc
            #x_pooled = self.iterative_pool(x_iterative.view(x_iterative.size(0), -1, 1)).squeeze(2)

            x_trans = self.transition_fc(x_iterative)

            # Stop decision based on the current state
            tensor_stop = torch.sigmoid(self.stop_decision_layer(x_trans))
            stop_decision = tensor_stop > 0.5

            # Update the mask - False for items that should stop
            mask = mask & ~stop_decision.squeeze()

            # Use the mask to selectively apply updates
            x = torch.where(mask.unsqueeze(1), x_trans, x)

            if not mask.any():
                break  # Exit if all items have met their stop condition

        # Processing using the last state of x for each item
        if random.random() < 0.001:
            print(i)

        x_final = F.relu(self.fc2(x))
        x_final = self.fc3(x_final)

        return x_final


cnn_with_iterative_stopnet = CNNWithIterativeStopNet()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn_with_iterative_stopnet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_with_iterative_stopnet.parameters(), lr=0.0036, momentum=0.91)

num_epochs = 20  # Define the number of epochs
N = 64

for epoch in range(num_epochs):  # Outer loop over epochs
    accumulated_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()  # Move zero_grad here
        outputs = cnn_with_iterative_stopnet(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        accumulated_loss += loss.item()

        if (i + 1) % N == 0:  # N is still 64; this condition is true every iteration with batch_size=64
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}], Loss: {accumulated_loss / N:.3f}')
            if( (i + 1) % (N*5) == 0):
              print(f"accuracy {eval(cnn_with_iterative_stopnet)}")
            accumulated_loss = 0.0  # Reset for the next set of N samples

print('Finished Training')







In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

class STEFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return (input > 0.5).float()

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output

ste = STEFunction.apply
import torch
import torch.nn as nn
import torch.nn.functional as F

def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

class ImageTransformerWithStopNet(nn.Module):
    def __init__(self, num_classes=10, d_model=512, nhead=8, num_layers=1, dim_feedforward=2048):
        super(ImageTransformerWithStopNet, self).__init__()

        self.embedding = nn.Linear(32*32*3, d_model)  # Assuming a 32x32x3 input image
        self.transformer_block = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward
        )
        self.stop_decision_layer = nn.Linear(d_model, 1)  # Stop decision layer
        self.fc = nn.Linear(d_model, num_classes)  # Final classifier

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add a sequence dimension

        mask = torch.ones(x.size(0), 1, dtype=torch.bool, device=x.device)  # Initialize mask

        max_iterations = 3  # Define how many times to reuse the encoder layer
        for _ in range(max_iterations):
            x = self.transformer_block(x)

            # Apply stop condition
            stop_decision = torch.sigmoid(self.stop_decision_layer(x[:, 0]))
            stop = (stop_decision > 0.5).squeeze()

            # Update mask based on stop decision
            mask = mask & ~stop.unsqueeze(1)

            # If all instances in the batch decide to stop, break the loop
            if not mask.any():
                break

        x = x.squeeze(1)  # Remove sequence dimension
        x_final = self.fc(x)  # Final classification layer

        return x_final


def _mp_fn(rank, flags):
    # Sets the default tensor type to float to avoid issues with mixed types
    torch.set_default_tensor_type('torch.FloatTensor')

    # Device configuration
    device = xm.xla_device()

    # Assuming you have defined your train dataset and transform
    # Define your dataset and DataLoader here. Example for CIFAR-10:
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True
    )
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=flags['batch_size'],
                                               sampler=train_sampler,
                                               num_workers=flags['num_workers'],
                                               drop_last=True)

    # Initialize model, loss, and optimizer
    model = ImageTransformerWithStopNet(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=flags['lr'], momentum=flags['momentum'])

    # Training loop
    num_epochs = flags['num_epochs']
    for epoch in range(num_epochs):
        para_loader = pl.ParallelLoader(train_loader, [device])
        xm.master_print(f'Starting epoch {epoch+1}/{num_epochs}')

        model.train()
        for inputs, labels in para_loader.per_device_loader(device):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            xm.optimizer_step(optimizer)

            # Optional: Print loss every N steps
            if xm.is_master_ordinal():
                xm.master_print(f'Step Loss: {loss.item()}')

        # Optional: Print training loss/accuracy or perform evaluation every epoch
        test_accuracy = evaluate(model, test_loader, device)
        xm.master_print(f'Epoch {epoch+1} Test Accuracy: {test_accuracy:.2f}%')

    # Additional logging or actions at the end of training
    if xm.is_master_ordinal():
        xm.master_print('Finished Training')

# Flags for configuration
flags = {
    'batch_size': 64,
    'num_workers': 2,
    'lr': 0.0036,
    'momentum': 0.91,
    'num_epochs': 10
}

# Start training process
xmp.spawn(_mp_fn, args=(flags,), nprocs=1)